# AULA 4

Modificar e aprimorar o Pipeline implementado na Aula 3, explorando novas técnicas de processamento e comparação de modelos.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Passo 1: Carregar o dataset
file_path = "[DADOS]ANO2C1B4S26A3A4_diabet1.csv"  # Nome do arquivo deve estar correto
df = pd.read_csv(file_path)

# Passo 2: Ajustar os nomes das colunas para evitar erros
df.columns = df.columns.str.strip().str.lower()

# Definir colunas numéricas para o Pipeline
num_features = ["glucose", "bloodpressure", "bmi", "insulin", "age", "diabetespedigreefunction"]

# Passo 3: Criar um Pipeline para o Pré-processamento
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),  # Preenchendo valores ausentes com a média
    ('scaler', StandardScaler())  # Normalizando os dados
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features)
])

# Passo 4: Divisão dos dados entre treino e teste
X = df.drop(columns=["outcome"])
y = df["outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Passo 5: Definir os modelos a serem comparados
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=2000)
}

# Passo 6: Comparar Modelos dentro do Pipeline
best_model = None
best_accuracy = 0
best_model_name = ""

for model_name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"\nModelo: {model_name}")
    print("Acurácia:", accuracy)
    print(classification_report(y_test, y_pred))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = pipeline
        best_model_name = model_name

print(f"\nMelhor modelo: {best_model_name} com acurácia de {best_accuracy}")

# Passo 7: Ajuste de Hiperparâmetros usando GridSearchCV
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
}

if best_model_name == 'RandomForest':
    grid_search = GridSearchCV(
        Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        param_grid,
        cv=5
    )

    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print("\nMelhores hiperparâmetros:", grid_search.best_params_)

# Passo 8: Salvar o melhor modelo
joblib.dump(best_model, "melhor_modelo_diabetes.pkl")

print("\nProcesso concluído! O melhor modelo foi salvo e pode ser utilizado para previsões futuras.")



Modelo: RandomForest
Acurácia: 0.7532467532467533
              precision    recall  f1-score   support

           0       0.83      0.78      0.80        99
           1       0.64      0.71      0.67        55

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.74       154
weighted avg       0.76      0.75      0.76       154


Modelo: LogisticRegression
Acurácia: 0.7597402597402597
              precision    recall  f1-score   support

           0       0.80      0.83      0.82        99
           1       0.67      0.64      0.65        55

    accuracy                           0.76       154
   macro avg       0.74      0.73      0.74       154
weighted avg       0.76      0.76      0.76       154


Melhor modelo: LogisticRegression com acurácia de 0.7597402597402597

Processo concluído! O melhor modelo foi salvo e pode ser utilizado para previsões futuras.
